In [1]:
import Pkg; Pkg.activate(@__DIR__); Pkg.instantiate()

  Activating environment at `/mnt/064AC6424AC62E6D/git_workspace/flyhopper/scripts/Julia/Project.toml`


In [2]:
using RigidBodyDynamics
using LinearAlgebra
using MeshCatMechanisms
using MeshCat
using StaticArrays

WebIO._IJuliaInit()

In [62]:
l1 = 1.0
l2 = 1.0
lc1 = l1/2
lc2 = l2/2
m1 = 1.0
m2 = 1.0
g = 9.81
I1 = m1*(l1^2)
I2 = m2*(l2^2)

9.81

In [63]:
world = RigidBody{Float64}("world")
doublependulum = Mechanism(world; gravity = SVector(0, 0, g))

Spanning tree:
Vertex: world (root)
No non-tree joints.

In [64]:
curdir = pwd()
urdfpath = joinpath(curdir, "../../res/flyhopper_mockup/urdf/flyhopper_mockup_jl.urdf")
doublependulum = parse_urdf(urdfpath, floating=false)
state = MechanismState(doublependulum)

MechanismState{Float64, Float64, Float64, …}(…)

In [154]:
function f_1(x)
    # double pendulum dynamics
    
    q1 = x[1]
    q1d = x[2]
    q2 = x[3]
    q2d = x[4]
    
    # Mass matrix
    M = zeros(2, 2)
    M[1, 1] = l1^2 * m1 + l1^2 *m2 + 2*l1*l2*m2*cos(q2) + l2^2 *m2
    M[1, 2] = l1*l2*m2*cos(q2) + l2^2 * m2
    M[2, 1] = l1*l2*m2*cos(q2) + l2^2 * m2
    M[2, 2] = l2^2 * m2
    
    
    # Coriolis term
    C = zeros(2)
    z = l1*l2*m2
    C[1] = -2*z*q1d*q2d*sin(q2) -z*(q2d^2)*sin(q2)
    C[2] = z*(q1d^2)*sin(q2)
    
    # Gravity term
    G = zeros(2)
    G[1] = g*l1*m1*cos(q1) + g*l1*m2*cos(q1) + g*l2*m2*cos(q1 + q2)
    G[2] = g*l2*m2*cos(q1 + q2)
      
    qd = zeros(2)
    qd[1] = q1d
    qd[2] = q2d
    
    qdd = M\(-G - C)
    q1dd = qdd[1]
    q2dd = qdd[2]
    
    ẋ = zeros(4)
    ẋ[1] = q1d # q1 dot
    ẋ[2] = q1dd # q1 double dot
    ẋ[3] = q2d # q2 dot
    ẋ[4] = q2dd # q2 double dot
    
    return ẋ
end

f_1 (generic function with 1 method)

In [155]:
# stolen from https://github.com/RoboticExplorationLab/RobotZoo.jl/blob/master/src/acrobot.jl

function f_2(x)
    u = [0, 0]
    g = 9.81
    # m1,m2 = model.m
    # l1,l2 = l1, l2
    J1,J2 = I1, I2
    θ1,    θ2    = x[1], x[3]
    θ1dot, θ2dot = x[2], x[4]
    s1,c1 = sincos(θ1)
    s2,c2 = sincos(θ2)
    c12 = cos(θ1 + θ2)

    # mass matrix
    m11 = m1*l1^2 + J1 + m2*(l1^2 + l2^2 + 2*l1*l2*c2) + J2
    m12 = m2*(l2^2 + l1*l2*c2 + J2)
    m22 = l2^2*m2 + J2
    M = @SMatrix [m11 m12; m12 m22]

    # bias term
    tmp = l1*l2*m2*s2
    b1 = -(2 * θ1dot * θ2dot + θ2dot^2)*tmp
    b2 = tmp * θ1dot^2
    B = @SVector [b1, b2]

    # friction
    c = 0
    C = @SVector [c*θ1dot, c*θ2dot]

    # gravity term
    g1 = ((m1 + m2)*l2*c1 + m2*l2*c12) * g
    g2 = m2*l2*c12*g
    G = @SVector [g1, g2]

    # equations of motion
    τ = @SVector [0, u[1]]
    θddot = M\(τ - B - G - C)
    return @SVector [θ1dot, θddot[1], θ2dot, θddot[2]]
end


f_2 (generic function with 1 method)

In [156]:
function rk4_step(f,xk,h)
    
    #TODO: implement rk4
    # xn = zeros(length(xk))
    
    a = [0 0 0 0; 
         0.5 0 0 0; 
         0 0.5 0 0; 
         0 0 1 0]
    b = [1/6, 1/3, 1/3, 1/6]
    f1 = f(xk)
    f2 = f(xk .+ h*a[2, 1]*f1)
    f3 = f(xk .+ h*a[3, 1]*f1 .+ h*a[3, 2]*f2)
    f4 = f(xk .+ h*a[4, 1]*f1 .+ h*a[4, 2]*f2 .+ h*a[4, 3]*f3)
    xn = xk .+ h*(b[1]*f1 .+ b[2]*f2 .+ b[3]*f3 .+ b[4]*f4)
    
    return xn
end

rk4_step (generic function with 1 method)

In [157]:
Tf = 10.0
h = 0.001 #20 Hz
N = Int(floor(Tf./h + 1))
thist = h.*Array(0:(N-1));

In [158]:
function simulate!(xtraj, N)
    for k = 1:(N-1)
        xtraj[:,k+1] .= rk4_step(f_1, xtraj[:,k], h)
    end
end

simulate! (generic function with 1 method)

In [159]:
x0 = [-150*(pi/180); 0.0; 120*(pi/180); 0.0]
xtraj = zeros(4,N)
xtraj[:,1] = x0;

In [160]:
simulate!(xtraj, N)

In [161]:
q1 = -xtraj[1, :] .- 150*(pi/180)
q2 = -xtraj[3, :] .+ 120*(pi/180)
qs = convert(AbstractArray{Float64, 2}, [q1 q2]) 
ts = convert(AbstractVector{Float64}, thist) # AbstractVector(thist)
q_array = [ qs[i,:] for i in 1:size(qs,1)] 

10001-element Vector{Vector{Float64}}:
 [0.0, 0.0]
 [-3.6410169981238028e-6, 6.068369128531259e-6]
 [-1.4564053171461921e-5, 2.4273541391117703e-5]
 [-3.276906406091129e-5, 5.461571142095778e-5]
 [-5.825597558306583e-5, 9.709520361678159e-5]
 [-9.102468405153274e-5, 0.00015171247215750583]
 [-0.000131075056209351, 0.00021846810102399417]
 [-0.00017840692926807122, 0.0002973628040257026]
 [-0.00023302011095749364, 0.0003883974248326538]
 [-0.00029491437958251154, 0.0004915729370158495]
 [-0.0003640894840901687, 0.0006068904440912348]
 [-0.00044054514414559875, 0.0007343511795712132]
 [-0.000524281050216846, 0.0008739565070219335]
 ⋮
 [-0.1791661381924734, 0.37287477387810375]
 [-0.18058274503312832, 0.3762937530627344]
 [-0.18200822697070418, 0.3797296819223941]
 [-0.1834426028733609, 0.38318261985683577]
 [-0.18488589186703352, 0.3866526269149624]
 [-0.18633811333927497, 0.3901397638026558]
 [-0.18779928694314707, 0.39364409189072735]
 [-0.18926943260116325, 0.39716567322299245]
 [-0.1

In [162]:
# mvis = MechanismVisualizer(doublependulum, Skeleton(randomize_colors=true, inertias=false));
mvis = MechanismVisualizer(doublependulum, URDFVisuals(urdfpath));

render(mvis)

┌ Info: MeshCat server started. You can open the visualizer by visiting the following URL in your browser:
│ http://127.0.0.1:8711
└ @ MeshCat /home/ben/.julia/packages/MeshCat/GlCMx/src/visualizer.jl:73


MeshCat.DisplayedVisualizer(MeshCat.CoreVisualizer(MeshCat.SceneTrees.SceneNode(nothing, nothing, Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}("meshcat" => MeshCat.SceneTrees.SceneNode(nothing, nothing, Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}("world" => MeshCat.SceneTrees.SceneNode(nothing, nothing, Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}("base_link" => MeshCat.SceneTrees.SceneNode(nothing, UInt8[0x83, 0xa6, 0x6d, 0x61, 0x74, 0x72, 0x69, 0x78, 0xc7, 0x40  …  0x2f, 0x62, 0x61, 0x73, 0x65, 0x5f, 0x6c, 0x69, 0x6e, 0x6b], Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}("geometry_1" => MeshCat.SceneTrees.SceneNode(UInt8[0x83, 0xa6, 0x6f, 0x62, 0x6a, 0x65, 0x63, 0x74, 0x84, 0xaa  …  0x67, 0x65, 0x6f, 0x6d, 0x65, 0x74, 0x72, 0x79, 0x5f, 0x31], UInt8[0x83, 0xa6, 0x6d, 0x61, 0x74, 0x72, 0x69, 0x78, 0xc7, 0x40  …  0x67, 0x65, 0x6f, 0x6d, 0x65, 0x74, 0x72, 0x79, 0x5f, 0x31], Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}()))), "Link 1" => MeshCat.SceneTrees.SceneNode(nothing, UInt8[0x83, 0xa6, 0x6d, 0x61, 0x74, 0x72, 0x69, 0x78, 0xc7, 0x40  …  0x72, 0x6c, 0x64, 0x2f, 0x4c, 0x69, 0x6e, 0x6b, 0x20, 0x31], Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}("after_Joint 1" => MeshCat.SceneTrees.SceneNode(nothing, UInt8[0x83, 0xa6, 0x6d, 0x61, 0x74, 0x72, 0x69, 0x78, 0xc7, 0x40  …  0x65, 0x72, 0x5f, 0x4a, 0x6f, 0x69, 0x6e, 0x74, 0x20, 0x31], Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}("geometry_2" => MeshCat.SceneTrees.SceneNode(UInt8[0x83, 0xa6, 0x6f, 0x62, 0x6a, 0x65, 0x63, 0x74, 0x84, 0xaa  …  0x67, 0x65, 0x6f, 0x6d, 0x65, 0x74, 0x72, 0x79, 0x5f, 0x32], UInt8[0x83, 0xa6, 0x6d, 0x61, 0x74, 0x72, 0x69, 0x78, 0xc7, 0x40  …  0x67, 0x65, 0x6f, 0x6d, 0x65, 0x74, 0x72, 0x79, 0x5f, 0x32], Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}()))), "Link 2" => MeshCat.SceneTrees.SceneNode(nothing, UInt8[0x83, 0xa6, 0x6d, 0x61, 0x74, 0x72, 0x69, 0x78, 0xc7, 0x40  …  0x6b, 0x20, 0x31, 0x2f, 0x4c, 0x69, 0x6e, 0x6b, 0x20, 0x32], Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}("after_Joint 2" => MeshCat.SceneTrees.SceneNode(nothing, UInt8[0x83, 0xa6, 0x6d, 0x61, 0x74, 0x72, 0x69, 0x78, 0xc7, 0x40  …  0x65, 0x72, 0x5f, 0x4a, 0x6f, 0x69, 0x6e, 0x74, 0x20, 0x32], Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}("geometry_3" => MeshCat.SceneTrees.SceneNode(UInt8[0x83, 0xa6, 0x6f, 0x62, 0x6a, 0x65, 0x63, 0x74, 0x84, 0xaa  …  0x67, 0x65, 0x6f, 0x6d, 0x65, 0x74, 0x72, 0x79, 0x5f, 0x33], UInt8[0x83, 0xa6, 0x6d, 0x61, 0x74, 0x72, 0x69, 0x78, 0xc7, 0x40  …  0x67, 0x65, 0x6f, 0x6d, 0x65, 0x74, 0x72, 0x79, 0x5f, 0x33], Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}()))))))))))))), Set{Any}(), ip"127.0.0.1", 8711))

In [163]:
# set_configuration!(mvis, [x0[1]-150*(pi/180), x0[3]+120*(pi/180)])
set_configuration!(mvis, [-x0[1]-150*(pi/180), -x0[3]+120*(pi/180)])
# set_configuration!(mvis, [0, 0])

In [164]:
# Now we can simply call `simulate`, which will return a tuple consisting of:
# * simulation times (a `Vector` of numbers)
# * joint configuration vectors (a `Vector` of `Vector`s)
# * joint velocity vectors (a `Vector` of `Vector`s)

# MeshCatMechanisms.animate(mvis, ts, qs; realtimerate = 1.);
animation = Animation(mvis, ts, -q_array)
setanimation!(mvis, animation)